# C Function Management and Registration

## Author: Zach Etienne

## Exploring how NRPy represents, formats, and registers C functions. This notebook walks through constructing `CFunction` instances, inspecting their generated prototypes and bodies, and registering them with `register_CFunction`. Along the way we highlight how to control descriptions, includes, subdirectories, coordinate system wrappers, and Einstein Toolkit metadata.

### Required reading if you are unfamiliar with programming or [computer algebra systems](https://en.wikipedia.org/wiki/Computer_algebra_system). Otherwise, use for reference; you should be able to pick up the syntax as you follow the tutorial.
+ **[Python Tutorial](https://docs.python.org/3/tutorial/index.html)**
+ **[SymPy Tutorial](http://docs.sympy.org/latest/tutorial/intro.html)**  (SymPy is not required for `CFunction` itself, but it is often used to generate the bodies.)

### NRPy Source Code for this module:  
* [c_function.py](../edit/c_function.py)

# Table of Contents

The module is organized as follows:

1. [Step 1](#Step-1:-Initialize-core-Python/NRPy-modules-for-CFunction): Initialize core Python/NRPy modules for `CFunction`
1. [Step 2](#Step-2:-Construct-your-first-CFunction-instance): Construct your first `CFunction` instance
1. [Step 3](#Step-3:-Descriptions,-comments,-and-prefix_with_star): Descriptions, comments, and `prefix_with_star`
1. [Step 4](#Step-4:-Controlling-function-prototypes-and-decorators): Controlling function prototypes and decorators
1. [Step 5](#Step-5:-Includes,-prefunc,-and-postfunc-snippets): Includes, `prefunc`, and `postfunc` snippets
1. [Step 6](#Step-6:-Subdirectories-and-subdirectory_depth-helpers): Subdirectories and `subdirectory_depth` helpers
1. [Step 7](#Step-7:-Registering-functions-with-register_CFunction): Registering functions with `register_CFunction`
1. [Step 8](#Step-8:-Coordinate-system-specific-wrapper-functions): Coordinate system specific wrapper functions
1. [Step 9](#Step-9:-Einstein-Toolkit-metadata-and-scheduling-hooks): Einstein Toolkit metadata and scheduling hooks

# Step 1: Initialize core Python/NRPy modules for `CFunction`
### \[Back to [top](#Table-of-Contents)\]

`CFunction` and its helpers live in NRPy's `c_function` module. In this step we import the pieces we will use throughout the notebook and briefly inspect the most important attributes.

The key objects are:

- `CFunction`: an object that stores a single C function and all of its metadata.
- `CFunction_dict`: a global dictionary mapping function names to `CFunction` instances.
- `function_name_and_subdir_with_CoordSystem`: a helper that adjusts names and subdirectories for coordinate system specific wrappers.
- `register_CFunction`: a convenience routine that constructs a `CFunction` and stores it in `CFunction_dict`.

In [1]:
# Step 1: Initialize core Python/NRPy modules for CFunction

import os

# Import the C function tools from NRPy
import nrpy.c_function as cfunc

from nrpy.c_function import (
    CFunction,
    CFunction_dict,
    function_name_and_subdir_with_CoordSystem,
    register_CFunction,
)

print("Imported CFunction tools from nrpy.c_function.")
print("Initial CFunction_dict keys:", list(CFunction_dict.keys()))

Imported CFunction tools from nrpy.c_function.
Initial CFunction_dict keys: []


# Step 2: Construct your first `CFunction` instance
### \[Back to [top](#Table-of-Contents)\]

At minimum a `CFunction` needs three pieces of information:

- `name`: the C function name.
- `desc`: a short documentation string that will become a comment block.
- `body`: the C code placed between the function braces.

All other fields have sensible defaults. Once the object is created, its
- `function_prototype`
- `raw_function`
- `full_function`

strings are populated automatically.

The prototype is a single line (for example `void myfunc(int x);`), while the full function contains includes, description comments, and the body wrapped in braces.

Let us create a tiny function that prints a greeting and then inspect the generated strings.

In [2]:
# Step 2: Construct your first CFunction instance

hello_func = CFunction(
    desc="Minimal example: print a greeting from C.",
    name="hello_world",
    params="void",
    body='printf("Hello from CFunction!\\n");'
)

print("Prototype:")
print(hello_func.function_prototype)

print("\nRaw function (before clang-format):")
print(hello_func.raw_function)

print("\nFull function (after optional clang-format):")
print(hello_func.full_function)

Prototype:
void hello_world(void);

Raw function (before clang-format):
/**
 * Minimal example: print a greeting from C.
*/
void hello_world(void) {
printf("Hello from CFunction!\n");
} // END FUNCTION hello_world




Full function (after optional clang-format):
/**
 * Minimal example: print a greeting from C.
 */
void hello_world(void) { printf("Hello from CFunction!\n"); } // END FUNCTION hello_world



Below are some useful questions to ask yourself whenever you construct a new `CFunction`:

- Does the prototype have the return type and parameter list you expect?
- Is the description turned into a C comment that would make sense to another reader?
- Does the body compile as valid C once you drop it into a source file?

Because `CFunction` stores its text in regular Python strings, you can always print, write, or modify them further with standard Python tools if needed.

# Step 3: Descriptions, comments, and `prefix_with_star`
### \[Back to [top](#Table-of-Contents)\]

The `desc` field becomes a C style documentation block:

```c
/**
 * description line 1
 * description line 2
 */
```

To make this formatting consistent, `CFunction` uses the helper `prefix_with_star` internally. You can also call it directly if you want to see how your multi line comment will be transformed.

The rules are:

- Each line is prefixed with `" * "` while preserving indentation.
- Lines that already start with a star are normalized.
- Empty lines become a bare `" *"` in the comment block.

Let us experiment with a few input strings, and then see how they are embedded into a function comment.

In [3]:
# Step 3: Descriptions, comments, and prefix_with_star

multiline_desc = """
Short summary line.
  * Second line already uses a star.
    Third line has extra indentation.
"""

print("Original description text:")
print(multiline_desc)

print("\nAfter prefix_with_star formatting:")
print(CFunction.prefix_with_star(multiline_desc))

commented_func = CFunction(
    desc=multiline_desc,
    name="documented_example",
    params="void",
    body="/* implementation goes here */"
)

print("\nFull function showing the formatted comment block:")
print(commented_func.full_function)

Original description text:

Short summary line.
  * Second line already uses a star.
    Third line has extra indentation.


After prefix_with_star formatting:
 * Short summary line.
 *  * Second line already uses a star.
 *     Third line has extra indentation.

Full function showing the formatted comment block:
/**
 * Short summary line.
 *  * Second line already uses a star.
 *     Third line has extra indentation.
 */
void documented_example(void) { /* implementation goes here */ } // END FUNCTION documented_example



You can think of `desc` as the place to write a human readable explanation of what your function does:

- What are the main inputs and outputs?
- Are there any assumptions (for example that an array has length at least $N$)?
- Are there any side effects that a caller should know about?

The exact formatting of the stars is handled for you, so you can focus on writing clear text.

# Step 4: Controlling function prototypes and decorators
### \[Back to [top](#Table-of-Contents)\]

By default, `CFunction` creates a prototype of the form

```c
void name(params);
```

but often you want more control:

- `cfunc_type`: sets the return type (for example `int`, `double`, `void`, or a struct type).
- `params`: sets the parameter list inside the parentheses.
- `cfunc_decorators`: adds tokens before the return type (for example `static inline`, `CUDA_DEVICE`, or templates).

The decorators are stored with a trailing space when present, so that the generated prototype reads naturally.

In [4]:
# Step 4: Controlling function prototypes and decorators

add_func = CFunction(
    desc="Return the sum of two integers.",
    cfunc_decorators="static inline",
    cfunc_type="int",
    name="add_two_ints",
    params="const int a, const int b",
    body="return a + b;"
)

print("Prototype with decorators:")
print(add_func.function_prototype)

print("\nFull function definition:")
print(add_func.full_function)

# A second example without decorators, returning a double.
norm_func = CFunction(
    desc="Compute a simple 2D Euclidean norm squared, r2 = x*x + y*y.",
    cfunc_type="double",
    name="norm2_2d",
    params="const double x, const double y",
    body="return x * x + y * y;"
)

print("\nPrototype without decorators:")
print(norm_func.function_prototype)

Prototype with decorators:
static inline int add_two_ints(const int a, const int b);

Full function definition:
/**
 * Return the sum of two integers.
 */
static inline int add_two_ints(const int a, const int b) { return a + b; } // END FUNCTION add_two_ints


Prototype without decorators:
double norm2_2d(const double x, const double y);


When designing your own functions, it is helpful to sketch the desired prototype on paper first, then map each piece to `cfunc_type`, `name`, `params`, and `cfunc_decorators`:

- Prototype: `static inline double f(const double *in, double *out);`
- Corresponding `CFunction` fields:
  - `cfunc_decorators="static inline"`
  - `cfunc_type="double"`
  - `name="f"`
  - `params="const double *in, double *out"`

# Step 5: Includes, `prefunc`, and `postfunc` snippets
### \[Back to [top](#Table-of-Contents)\]

Real C functions almost always sit inside a larger file with:

- `#include` directives,
- macro definitions,
- helper code above or below the function body.

`CFunction` lets you attach these pieces directly to the function:

- `includes`: a list of header names. Items containing `<` are emitted as `#include <...>`, all others as `#include "..."`.
- `prefunc`: a chunk of text inserted above the function definition (after any includes).
- `postfunc`: a chunk of text inserted after the function definition.

In addition, the `include_CodeParameters_h` and `enable_simd` flags control whether a CodeParameters header is automatically included inside the function body. If `include_CodeParameters_h` is `True`, a header for code parameters is inserted near the top of the body, and a SIMD aware version is chosen when SIMD support is requested.

In [5]:
# Step 5: Includes, prefunc, and postfunc snippets

io_func = CFunction(
    includes=["<stdio.h>", "my_header.h"],
    prefunc="""
#ifdef USE_MY_HEADER
#define MY_FACTOR 2
#else
#define MY_FACTOR 1
#endif
""",
    desc="Print a scaled integer value.",
    cfunc_type="void",
    name="print_scaled",
    params="int value",
    body="""
const int scaled = value * MY_FACTOR;
printf("scaled = %d\\n", scaled);
""",
    postfunc="""
// Extra helper code could be added here, such as inline utilities
// that conceptually belong next to print_scaled.
"""
)

print("Full function including includes, prefunc, and postfunc:")
print(io_func.full_function)

# Demonstrate automatic inclusion of CodeParameters when requested.
params_func = CFunction(
    include_CodeParameters_h=True,
    enable_simd=False,
    desc="Example that needs global CodeParameters.",
    cfunc_type="void",
    name="use_parameters",
    params="void",
    body="""
/* Here you might access global CodeParameters defined elsewhere.
 * The appropriate header is automatically included at the top of the body.
 */
"""
)

print("\nFunction that automatically includes a CodeParameters header:")
print(params_func.full_function)

Full function including includes, prefunc, and postfunc:
#include "my_header.h"
#include <stdio.h>

#ifdef USE_MY_HEADER
#define MY_FACTOR 2
#else
#define MY_FACTOR 1
#endif

/**
 * Print a scaled integer value.
 */
void print_scaled(int value) {
  const int scaled = value * MY_FACTOR;
  printf("scaled = %d\n", scaled);
} // END FUNCTION print_scaled

// Extra helper code could be added here, such as inline utilities
// that conceptually belong next to print_scaled.


Function that automatically includes a CodeParameters header:
/**
 * Example that needs global CodeParameters.
 */
void use_parameters(void) {
#include "set_CodeParameters.h"
  /* Here you might access global CodeParameters defined elsewhere.
   * The appropriate header is automatically included at the top of the body.
   */
} // END FUNCTION use_parameters



A typical workflow is:

1. Decide which headers and macros your function needs.
2. Place macros or helper definitions that must appear immediately before the function into `prefunc`.
3. Reserve `postfunc` for closely related helpers that conceptually follow the main function.
4. Turn on `include_CodeParameters_h` when your function depends on global code parameters, letting `CFunction` decide which header variant to use.

# Step 6: Subdirectories and `subdirectory_depth` helpers
### \[Back to [top](#Table-of-Contents)\]

Each `CFunction` carries a `subdirectory` attribute describing where it should live in the generated source tree, relative to some root directory. This is a plain path string, such as

- `"."` for the current directory,
- `"evolution/sources/metric"` for a nested structure.

The static method `subdirectory_depth` counts how many path components are present, which is useful when deciding how many `"../"` pieces are needed to reach the project root.

The path is normalized, so redundant separators and trivial components like `"./"` are handled for you.

In [6]:
# Step 6: Subdirectories and subdirectory_depth helpers

nested_func = CFunction(
    subdirectory=os.path.join("evolution", "sources", "metric"),
    desc="Example that lives in a nested directory.",
    cfunc_type="void",
    name="nested_example",
    params="void",
    body="/* implementation omitted */"
)

print("Subdirectory stored on the object:")
print(nested_func.subdirectory)

print("\nDepth computed by subdirectory_depth for this subdirectory:")
print(CFunction.subdirectory_depth(nested_func.subdirectory))

print("\nSome additional subdirectory_depth examples:")
examples = [
    ".",
    "./",
    "coordinates",
    "coordinates/radial",
    "./output//diagnostics///",
]

for path in examples:
    print(f"  path={path!r} -> depth={CFunction.subdirectory_depth(path)}")

Subdirectory stored on the object:
evolution/sources/metric

Depth computed by subdirectory_depth for this subdirectory:
3

Some additional subdirectory_depth examples:
  path='.' -> depth=0
  path='./' -> depth=0
  path='coordinates' -> depth=1
  path='coordinates/radial' -> depth=2
  path='./output//diagnostics///' -> depth=2


Design tips:

- Use `subdirectory` to express how your generated C files should be organized hierarchically.
- Use `subdirectory_depth` when constructing relative include paths or other filesystem aware strings.
- Keep subdirectory names short but descriptive, for example `"initial_data"`, `"evolution"`, `"analysis"`.

# Step 7: Registering functions with `register_CFunction`
### \[Back to [top](#Table-of-Contents)\]

So far we have created standalone `CFunction` instances. In practice you often want to collect them in a central registry that other parts of NRPy can query. This role is played by:

- `CFunction_dict`: a dictionary mapping function names to `CFunction` objects.
- `register_CFunction`: a helper that constructs a `CFunction` and inserts it into `CFunction_dict`.

Important properties:

- Each function name must be unique inside `CFunction_dict`. Attempting to register the same name twice raises a `ValueError`.
- Most arguments to `register_CFunction` are passed directly through to the underlying `CFunction` constructor.
- Coordinate system wrappers are handled transparently (we will see that in the next step).

Let us walk through a small registry example.

In [7]:
# Step 7: Registering functions with register_CFunction

# For the purposes of this example, start with a clean registry.
CFunction_dict.clear()

register_CFunction(
    name="compute_answer",
    desc="Return a fixed integer answer.",
    cfunc_type="int",
    params="void",
    body="return 42;"
)

register_CFunction(
    name="scale_value",
    desc="Multiply an input integer by a factor.",
    cfunc_type="int",
    params="int value, int factor",
    body="return value * factor;"
)

print("Currently registered function names:")
for key in CFunction_dict:
    print(" -", key)

print("\nFull function text for compute_answer:")
print(CFunction_dict["compute_answer"].full_function)

# Demonstrate what happens when registering a duplicate name.
try:
    register_CFunction(
        name="compute_answer",
        desc="This second definition should fail.",
        cfunc_type="int",
        params="void",
        body="return 0;"
    )
except ValueError as exc:
    print("Caught ValueError when trying to register a duplicate name:")
    print(exc)

Currently registered function names:
 - compute_answer
 - scale_value

Full function text for compute_answer:
/**
 * Return a fixed integer answer.
 */
int compute_answer(void) { return 42; } // END FUNCTION compute_answer

Caught ValueError when trying to register a duplicate name:
Error: already registered compute_answer in CFunction_dict.


In a larger project, one common pattern is:

- Have each NRPy module call `register_CFunction` for the functions it defines.
- Later, iterate over `CFunction_dict` to write the full set of C source files to disk.
- Use the dictionary keys (the function names) as stable identifiers when building wrappers or schedules.

# Step 8: Coordinate system specific wrapper functions
### \[Back to [top](#Table-of-Contents)\]

In the BHaH infrastructure it is common to provide both:

- a generic wrapper function, for example `xx_to_Cart`, and
- one or more coordinate system specific implementations, such as a version specialized for `"SinhSpherical"`.

The helper

```python
function_name_and_subdir_with_CoordSystem(subdirectory, name, CoordSystem_for_wrapper_func)
```

returns the adjusted subdirectory and function name for a coordinate system specific version. When `CoordSystem_for_wrapper_func` is a nonempty string, both the subdirectory and the function name are modified in a predictable way.

`register_CFunction` uses this helper internally, so you can either:

- call `function_name_and_subdir_with_CoordSystem` directly, or
- pass `CoordSystem_for_wrapper_func` into `register_CFunction` and let it handle the bookkeeping.

Let us see both approaches in action.

In [8]:
# Step 8: Coordinate system specific wrapper functions

# Direct use of the helper:
subdir, wrapper_name = function_name_and_subdir_with_CoordSystem(
    os.path.join("."),
    "xx_to_Cart",
    "SinhSpherical"
)

print("Computed subdirectory and wrapper name from helper:")
print("  subdirectory:", subdir)
print("  wrapper name:", wrapper_name)

# Now let register_CFunction handle the coordinate system specific naming.
CFunction_dict.clear()

register_CFunction(
    subdirectory="coordinate_transforms",
    CoordSystem_for_wrapper_func="SinhSpherical",
    name="xx_to_Cart",
    desc="Transform reference coordinates xx to Cartesian coordinates in a SinhSpherical chart.",
    cfunc_type="void",
    params="const double *xx, double *xCart",
    body="""
/* Example body: fill xCart[] from xx[] using the sinh-spherical mapping.
 * In real code this would contain the actual transformation formulae.
 */
"""
)

print("\nKeys in CFunction_dict after registration:")
for key, func in CFunction_dict.items():
    print("  registered name:", key)
    print("  stored subdirectory:", func.subdirectory)

Computed subdirectory and wrapper name from helper:
  subdirectory: ./SinhSpherical
  wrapper name: xx_to_Cart__rfm__SinhSpherical

Keys in CFunction_dict after registration:
  registered name: xx_to_Cart__rfm__SinhSpherical
  stored subdirectory: coordinate_transforms/SinhSpherical


When you design coordinate system aware code:

- Use plain `name="xx_to_Cart"` for the conceptual operation.
- Use `CoordSystem_for_wrapper_func` to distinguish concrete realizations, such as `"SinhSpherical"` or `"Cartesian"`.
- Keep subdirectories aligned with these choices, so it is clear where each specialization lives on disk.

# Step 9: Einstein Toolkit metadata and scheduling hooks
### \[Back to [top](#Table-of-Contents)\]

For users targeting the Einstein Toolkit (ET), `CFunction` can carry additional metadata that drives:

- where and when the function is scheduled to run, and
- which `CodeParameters` it depends on.

The relevant fields are:

- `ET_thorn_name`: the thorn that owns this function.
- `ET_schedule_bins_entries`: a list of `(schedule_bin, schedule_entry)` tuples for `schedule.ccl`.
- `ET_current_thorn_CodeParams_used`: the list of parameter names from the current thorn used by this function.
- `ET_other_thorn_CodeParams_used`: the list of parameter names from other thorns used by this function.

These fields are stored on the `CFunction` object and can later be consumed by code that writes ET interface files.

In [9]:
# Step 9: Einstein Toolkit metadata and scheduling hooks

CFunction_dict.clear()

register_CFunction(
    name="ETK_example",
    desc="Demonstrate how ET metadata is attached to a CFunction.",
    cfunc_type="void",
    params="void",
    body="/* evolution or analysis code would go here */",
    ET_thorn_name="MyExampleThorn",
    ET_schedule_bins_entries=[
        ("CCTK_BASEGRID", "ETK_example"),
        ("CCTK_POSTSTEP", "ETK_example"),
    ],
    ET_current_thorn_CodeParams_used=["my_param1", "my_param2"],
    ET_other_thorn_CodeParams_used=["otherthorn::their_param"]
)

et_func = CFunction_dict["ETK_example"]

print("Function name:", et_func.name)
print("  ET_thorn_name:", et_func.ET_thorn_name)
print("  ET_schedule_bins_entries:", et_func.ET_schedule_bins_entries)
print("  ET_current_thorn_CodeParams_used:", et_func.ET_current_thorn_CodeParams_used)
print("  ET_other_thorn_CodeParams_used:", et_func.ET_other_thorn_CodeParams_used)

print("\nFull function text:")
print(et_func.full_function)

Function name: ETK_example
  ET_thorn_name: MyExampleThorn
  ET_schedule_bins_entries: [('CCTK_BASEGRID', 'ETK_example'), ('CCTK_POSTSTEP', 'ETK_example')]
  ET_current_thorn_CodeParams_used: ['my_param1', 'my_param2']
  ET_other_thorn_CodeParams_used: ['otherthorn::their_param']

Full function text:
/**
 * Demonstrate how ET metadata is attached to a CFunction.
 */
void ETK_example(void) { /* evolution or analysis code would go here */ } // END FUNCTION ETK_example



When integrating NRPy generated code into an Einstein Toolkit thorn, a typical pattern is:

- Use `register_CFunction` to attach ET metadata as you define each function.
- Later, iterate over the registered functions to generate `schedule.ccl` and `param.ccl` fragments.
- Keep the metadata close to the function definitions so that changes to one are unlikely to drift out of sync with the other.